**Table of contents**<a id='toc0_'></a>    
- [Dataset和DataLoader，以及训练参数。](#toc1_1_)    
    - [epoch，batch_size等参数](#toc1_1_1_)    
  - [torchvision](#toc1_2_)    
  - [学习率和优化器：](#toc1_3_)    
    - [学习率](#toc1_3_1_)    
    - [优化器](#toc1_3_2_)    
  - [损失函数](#toc1_4_)    
  - [卷积](#toc1_5_)    
  - [池化层](#toc1_6_)    
    - [最大池化](#toc1_6_1_)    
    - [平均池化](#toc1_6_2_)    
    - [反池化](#toc1_6_3_)    
  - [线性层](#toc1_7_)    
  - [激活函数层](#toc1_8_)    
  - [权重初始化](#toc1_9_)    
    - [激活函数与推荐初始化方法](#toc1_9_1_)    
  - [Hook 函数](#toc1_10_)    
  - [正则化](#toc1_11_)    
  - [归一化](#toc1_12_)    
  - [模型保存与加载](#toc1_13_)    
    - [基本方法](#toc1_13_1_)    
      - [(1) 保存整个模型（结构+参数）](#toc1_13_1_1_)    
      - [(2) 仅保存模型参数（推荐）](#toc1_13_1_2_)    
    - [带优化器的保存（用于恢复训练）](#toc1_13_2_)    
    - [跨设备加载](#toc1_13_3_)    
    - [注意事项](#toc1_13_4_)    
    - [完整示例代码](#toc1_13_5_)    
    - [常用文件结构](#toc1_13_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

我的 PyTorch 笔记和代码

## <a id='toc1_1_'></a>[Dataset和DataLoader，以及训练参数。](#toc0_)

Dataset: 自定义数据集，包括读取数据集、处理方式等
DataLoader: 加载数据集，将数据集装载为一个可迭代对象
Model: 定义模型，包括网络结构、参数等
Criterion: 定义损失函数，包括计算损失的公式
Criterion: 定义损失函数
Optimizer: 定义优化器
Trainer: 训练模型

### <a id='toc1_1_1_'></a>[epoch，batch_size等参数](#toc0_)
Epoch: 所有训练样本都已经输入到模型中，称为一个 Epoch
epoch 指定所有样本被训练几轮，决定了模型将看到整个数据集多少次，常用 50,100，太多会过拟合

Iteration: 一批样本输入到模型中，称为一个 Iteration
即迭代次数，完整训练一次所有样本需要的次数

Batchsize: 批大小，决定一个 iteration 有多少样本，也决定了一个 Epoch 有多少个 Iteration。

每次输入到模型上的样本数，一般为 16、32、64、128、256、512 等，太多了会爆显存，根据模型而已，有的 64 就会爆，修妖减小才行，大的 batchsize 训练更稳定，训练时间长，收敛效果好，小的 batchsize 训练更快，训练时间短，收敛效果差（噪声大）。

Iteration \* Batchsize = 一轮的样本总数，即一个 epoch 要训练多少个样本
Total Iterations = Number of Epochs × (Total Samples / Batch Size)
总轮次 = 总 Epoch 数 × (样本总数 / 批大小)


In [ ]:
# 训练模型典型流程
for epoch in range(num_epochs):  # 外层循环控制Epoch
    for batch_idx, (data, target) in enumerate(train_loader):  # 内层循环处理Iteration
        # 1. 将数据移动到设备（GPU/CPU）
        data, target = data.to(device), target.to(device)

        # 2. 前向传播
        output = model(data)

        # 3. 计算损失
        loss = criterion(output, target)

        # 4. 反向传播
        optimizer.zero_grad()  # 清空梯度
        loss.backward()  # 计算梯度

        # 5. 参数更新
        optimizer.step()

## <a id='toc1_2_'></a>[torchvision](#toc0_)

我们在安装 PyTorch 时，还安装了 torchvision，这是一个计算机视觉工具包。有 3 个主要的模块：
torchvision.transforms 模块提供了一些常用的图像预处理方法，如裁剪、缩放、旋转、翻转等。
torchvision.datasets: 里面包括常用数据集如 mnist、CIFAR-10、Image-Net 等
torchvision.models: 里面包括常用的模型如 AlexNet、VGG、ResNet、GoogleNet 等。

ps:torchvision 是计算机视觉库，提供常用的数据集和预处理方法

PyTorch 的主要模块有：
torch：核心模块，包含张量计算、自动求导、并行计算等功能。
torch.nn：神经网络模块，包含各种神经网络层、损失函数等。
torch.optim：优化器模块，包含常用的优化算法。
torch.utils.data：数据集模块，包含常用的数据集加载器。
torch.autograd：自动求导模块，包含自动求导引擎。


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# 设置训练集的数据增强和转化
train_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),  # 缩放
        transforms.RandomCrop(32, padding=4),  # 裁剪
        transforms.ToTensor(),  # 转为张量，同时归一化
        transforms.Normalize(norm_mean, norm_std),  # 标准化
    ]
)

# 设置验证集的数据增强和转化，不需要 RandomCrop
valid_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std),
    ]
)

## <a id='toc1_3_'></a>[学习率和优化器：](#toc0_)

### <a id='toc1_3_1_'></a>[学习率](#toc0_)

LR，控制每次参数更新的步长，是优化器调整模型权重的幅度，一般要前打后小，并且总体不能过大，否则会导致模型震荡。

常使用学习率衰减策略，包括余弦衰减、指数衰减、线性衰减等。

### <a id='toc1_3_2_'></a>[优化器](#toc0_)

优化器是指模型训练时使用的算法，包括SGD、Adam、Adagrad、Adadelta、RMSprop等。

SGD：随机梯度下降，是最常用的优化器，其基本思想是每次迭代时，随机选择一个样本，计算其梯度，然后更新模型参数。

Adam：Adam是一种基于动量的优化器，其基本思想是计算梯度的指数加权移动平均值，使得模型的更新方向更加准确。也是默认学习器，大多数时候已经足够好
（Adam的优点是能够自适应调整学习率，并且内置了动量，不能再手动指定动量）

AdamW：AdamW是Adam的变体，其在Adam的基础上，增加了权重衰减项，可以防止过拟合。

RMPprop：RMSprop是Adadelta的变体，其基本思想是对梯度的指数加权移动平均值，并对其进行平方根处理，使得更新更加稳定。用于RNN和LSTM。

## <a id='toc1_4_'></a>[损失函数](#toc0_)

MSE：均方误差损失，即L2损失函数，即预测值与真实值之差的平方的平均值，常用于回归问题，预测的是连续值。
$$MSE=\frac{1}{n}\sum_{i=1}^{n}(y_{i}-\hat{y}_{i})^{2}$$

MAE：平均绝对误差，即L1损失函数，即预测值与真实值之差的绝对值的平均值，常用于回归问题，预测的是连续值。
$$MAE=\frac{1}{n}\sum_{i=1}^{n}\left|y_{i}-\hat{y}_{i}\right|$$

CrossEntropyLoss：交叉熵损失函数：分类问题常用的损失函数，衡量两个概率分布之间的差异。
$$H(p,q)=-\sum_{i=1}^{n}p_{i}\log(q_{i})$$

其中，$p$和$q$分别是真实分布和预测分布，$n$是样本数。

CrossEntropyLoss和Softmax函数的结合：

CrossEntropyLoss函数的输入是预测的概率分布，而Softmax函数的输入是预测的类别分布。因此，我们可以将CrossEntropyLoss函数的输入经过Softmax函数转换为类别分布，从而得到最终的损失值。

$$\ell(x,y)=\frac{1}{n}\sum_{i=1}^{n}\log\left(\frac{\exp(x_{i,y})}{\sum_{j=1}^{k}\exp(x_{i,j})}\right)$$

创建一个模型

1.putorch中，模型的定义一般是继承nn.Module类，并实现__init__()方法和forward()方法。
2.在__init__()方法中，定义模型的层，例如卷积层、全连接层等。
3.在forward()方法中，对层进行拼接，实现模型的前向传播过程，即输入数据经过模型得到输出。

在拼接层时，可以用nn.Sequential()函数，它可以将多个层组合成一个模型。

In [5]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# 定义一个模型
class MyTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyTorch, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# 定义一个加入了nn.sequential的简单模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 3),
            nn.ReLU(),
            nn.Linear(3, 1)
        )

    def forward(self, x):
        return self.net(x)

net = MyModel()
print(net)
net = net(torch.randn(1, 2)) # 输入1个样本，2个特征
# net = net(torch.randn(2, 2)) # 输入2个样本，2个特征

print(net)

MyModel(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
  )
)
tensor([[-0.0501]], grad_fn=<AddmmBackward0>)


## <a id='toc1_5_'></a>[卷积](#toc0_)

二维卷积：Conv2d，
```python
nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros')
```

三维卷积：Conv3d，
```python
nn.Conv3d(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros')
```
包括输入通道数，输出通道数，卷积核大小，步长，填充，膨胀，组数，偏置，填充模式。
一般就指定前五个，输入，输出，卷积核大小，填充（0），步长（1），后面都是默认
步长为1是卷积，步长为2是下采样
参数说明：
in_channels	输入数据的通道数（如RGB图像=3，灰度图=1）

out_channels	卷积后输出的通道数（即卷积核的数量）

kernel_size	卷积核的尺寸（整数或元组，如 3 或 (3,5)）

stride	卷积核移动的步长（控制输出尺寸的缩小比例）

padding	输入边缘填充的像素数（保持空间分辨率）

dilation	卷积核的膨胀率（扩大感受野，如空洞卷积）

groups	分组卷积的组数（in_channels和out_channels需能被整除）

bias	是否添加可学习的偏置项（True/False）

padding_mode	填充方式（'zeros'、'reflect'、'replicate'）

这里不考虑空洞卷积，假设输入图片大小为 $ I \times I$，卷积核大小为 $k \times k$，stride 为 $s$，padding 的像素数为 $p$，图片经过卷积之后的尺寸 $ O $ 如下：

$O = \displaystyle\frac{I -k + 2 \times p}{s} +1$

下面例子的输入图片大小为 $5 \times 5$，卷积大小为 $3 \times 3$，stride 为 1，padding 为 0，所以输出图片大小为 $\displaystyle\frac{5 -3 + 2 \times 0}{1} +1 = 3$。

因为padding是填两层，左右上下两个方向都填充了0。

## <a id='toc1_6_'></a>[池化层](#toc0_)

池化的作用则体现在降采样：保留显著特征、降低特征维度，增大kernel的感受野。 

另外一点值得注意：pooling也可以提供一些旋转不变性。 池化层可对提取到的特征信息进行降维，一方面使特征图变小，简化网络计算复杂度并在一定程度上避免过拟合的出现；一方面进行特征压缩，提取主要特征。

有最大池化和平均池化两张方式。

### <a id='toc1_6_1_'></a>[最大池化](#toc0_)

nn.MaxPool2d()

nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
这个函数的功能是进行 2 维的最大池化，主要参数如下：

kernel_size：池化核尺寸

stride：步长，通常与 kernel_size 一致

padding：填充宽度，主要是为了调整输出的特征图大小，一般把 padding 设置合适的值后，保持输入和输出的图像尺寸不变。

dilation：池化间隔大小，默认为1。常用于图像分割任务中，主要是为了提升感受野

ceil_mode：默认为 False，尺寸向下取整。为 True 时，尺寸向上取整

return_indices：为 True 时，返回最大池化所使用的像素的索引，这些记录的索引通常在反最大池化时使用，把小的特征图反池化到大的特征图时，每一个像素放在哪个位置。



下面是最大池化的代码：

```python
import os
import torch
import torch.nn as nn
from torchvision import transforms
from matplotlib import pyplot as plt
from PIL import Image
from common_tools import transform_invert, set_seed

set_seed(1)  # 设置随机种子

# ================================= load img ==================================
path_img = os.path.join(os.path.dirname(os.path.abspath(__file__)), "imgs/lena.png")
img = Image.open(path_img).convert('RGB')  # 0~255

# convert to tensor
img_transform = transforms.Compose([transforms.ToTensor()])
img_tensor = img_transform(img)
img_tensor.unsqueeze_(dim=0)    # C*H*W to B*C*H*W

# ============= create convolution layer ===============

# ================ maxpool
flag = 1
# flag = 0
if flag:
    maxpool_layer = nn.MaxPool2d((2, 2), stride=(2, 2))   # input:(i, o, size) weights:(o, i , h, w)
    img_pool = maxpool_layer(img_tensor)

print("池化前尺寸:{}\n池化后尺寸:{}".format(img_tensor.shape, img_pool.shape))
img_pool = transform_invert(img_pool[0, 0:3, ...], img_transform)
img_raw = transform_invert(img_tensor.squeeze(), img_transform)
plt.subplot(122).imshow(img_pool)
plt.subplot(121).imshow(img_raw)
plt.show()

池化前尺寸:torch.Size([1, 3, 512, 512])
池化后尺寸:torch.Size([1, 3, 256, 256])
```
### <a id='toc1_6_2_'></a>[平均池化](#toc0_)

```python
nn.AvgPool2d()

torch.nn.AvgPool2d(kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None)
这个函数的功能是进行 2 维的平均池化，主要参数如下：

kernel_size：池化核尺寸

stride：步长，通常与 kernel_size 一致

padding：填充宽度，主要是为了调整输出的特征图大小，一般把 padding 设置合适的值后，保持输入和输出的图像尺寸不变。

dilation：池化间隔大小，默认为1。常用于图像分割任务中，主要是为了提升感受野

ceil_mode：默认为 False，尺寸向下取整。为 True 时，尺寸向上取整

count_include_pad：在计算平均值时，是否把填充值考虑在内计算

divisor_override：除法因子。在计算平均值时，分子是像素值的总和，分母默认是像素值的个数。如果设置了 divisor_override，把分母改为 divisor_override。

img_tensor = torch.ones((1, 1, 4, 4))
avgpool_layer = nn.AvgPool2d((2, 2), stride=(2, 2))
img_pool = avgpool_layer(img_tensor)
print("raw_img:\n{}\npooling_img:\n{}".format(img_tensor, img_pool))
输出如下：

raw_img:
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])
pooling_img:
tensor([[[[1., 1.],
          [1., 1.]]]])
加上divisor_override=3后，输出如下：

raw_img:
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])
pooling_img:
tensor([[[[1.3333, 1.3333],
          [1.3333, 1.3333]]]])
nn.MaxUnpool2d()

```

### <a id='toc1_6_3_'></a>[反池化](#toc0_)

```python
nn.MaxUnpool2d(kernel_size, stride=None, padding=0)
功能是对二维信号（图像）进行最大值反池化，主要参数如下：

kernel_size：池化核尺寸

stride：步长，通常与 kernel_size 一致

padding：填充宽度

代码如下：

# pooling
img_tensor = torch.randint(high=5, size=(1, 1, 4, 4), dtype=torch.float)
maxpool_layer = nn.MaxPool2d((2, 2), stride=(2, 2), return_indices=True)
img_pool, indices = maxpool_layer(img_tensor)

# unpooling
img_reconstruct = torch.randn_like(img_pool, dtype=torch.float)
maxunpool_layer = nn.MaxUnpool2d((2, 2), stride=(2, 2))
img_unpool = maxunpool_layer(img_reconstruct, indices)

print("raw_img:\n{}\nimg_pool:\n{}".format(img_tensor, img_pool))
print("img_reconstruct:\n{}\nimg_unpool:\n{}".format(img_reconstruct, img_unpool))
输出如下：

# pooling
img_tensor = torch.randint(high=5, size=(1, 1, 4, 4), dtype=torch.float)
maxpool_layer = nn.MaxPool2d((2, 2), stride=(2, 2), return_indices=True)
img_pool, indices = maxpool_layer(img_tensor)

# unpooling
img_reconstruct = torch.randn_like(img_pool, dtype=torch.float)
maxunpool_layer = nn.MaxUnpool2d((2, 2), stride=(2, 2))
img_unpool = maxunpool_layer(img_reconstruct, indices)

print("raw_img:\n{}\nimg_pool:\n{}".format(img_tensor, img_pool))
print("img_reconstruct:\n{}\nimg_unpool:\n{}".format(img_reconstruct, img_unpool))

```

## <a id='toc1_7_'></a>[线性层](#toc0_)


线性层又称为全连接层，其每个神经元与上一个层所有神经元相连，实现对前一层的线性组合或线性变换。其要求输入是一位向量，因此往往会将多维数据展开成一维的，因此会丢失空间结构。加上无法聚焦局部区域，空间结构混乱，一张图从左往右和从右往左会得到不同的向量表示，但是卷积层会有权值共享维持平移不变形。其y=Wx+b的形式，其中W是权重矩阵，b是偏置项。

```python

nn.Linear(in_features, out_features, bias=True) # NOTE 这个居然就是全连接层，2025.6.29,第一次知道

代码如下：

inputs = torch.tensor([[1., 2, 3]])
linear_layer = nn.Linear(3, 4)
linear_layer.weight.data = torch.tensor([[1., 1., 1.],
[2., 2., 2.],
[3., 3., 3.],
[4., 4., 4.]])

linear_layer.bias.data.fill_(0.5)
output = linear_layer(inputs)
print(inputs, inputs.shape)
print(linear_layer.weight.data, linear_layer.weight.data.shape)
print(output, output.shape)
输出为：

复制
tensor([[1., 2., 3.]]) torch.Size([1, 3])
tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.],
        [4., 4., 4.]]) torch.Size([4, 3])
tensor([[ 6.5000, 12.5000, 18.5000, 24.5000]], grad_fn=<AddmmBackward>) torch.Size([1, 4])

```

## <a id='toc1_8_'></a>[激活函数层](#toc0_)

激活函数层的作用是对神经网络的输出进行非线性变换，使得输出值不再是线性的，从而使得神经网络能够拟合非线性数据。
假设第一个隐藏层为：$H_{1}=X \times W_{1}$，第二个隐藏层为：$H_{2}=H_{1} \times W_{2}$，输出层为：

$ \begin{aligned} \text { Out } \boldsymbol{p} \boldsymbol{u} \boldsymbol{t} &=\boldsymbol{H}{2} * \boldsymbol{W}{3} \ &=\boldsymbol{H}{1} * \boldsymbol{W}{2} \boldsymbol{W}_{3} \ &=\boldsymbol{X} (\boldsymbol{W}{1} *\boldsymbol{W}{2} \boldsymbol{W}_{3}) \ &=\boldsymbol{X} {W} \end{aligned} $

如果没有非线性变换，由于矩阵乘法的结合性，多个线性层的组合等价于一个线性层。

激活函数对特征进行非线性变换，赋予了多层神经网络具有深度的意义。下面介绍一些激活函数层。

nn.Sigmoid
计算公式：$y=\frac{1}{1+e^{-x}}$

梯度公式：$y^{\prime}=y *(1-y)$

特性：

输出值在(0,1)，符合概率

导数范围是 [0, 0.25]，容易导致梯度消失

输出为非 0 均值，破坏数据分布

nn.tanh
计算公式：$y=\frac{\sin x}{\cos x}=\frac{e^{x}-e^{-x}}{e^{-}+e^{-x}}=\frac{2}{1+e^{-2 x}}+1$

梯度公式：$y^{\prime}=1-y^{2}$

特性：

输出值在(-1, 1)，数据符合 0 均值

导数范围是 (0,1)，容易导致梯度消失

nn.ReLU(修正线性单元)
计算公式：$y=max(0, x)$

梯度公式：$y' = \begin{cases} 1 & x > 0 \\ 0 & x < 0 \\ \text{undefined} & x=0 \end{cases}$

特性：

输出值均为正数，负半轴的导数为 0，容易导致死神经元

导数是 1，缓解梯度消失，但容易引发梯度爆炸


针对 RuLU 会导致死神经元的缺点，出现了下面 3 种改进的激活函数。

nn.LeakyReLU
有一个参数negative_slope：设置负半轴斜率

nn.PReLU
有一个参数init：设置初始斜率，这个斜率是可学习的

nn.RReLU
R 是 random 的意思，负半轴每次斜率都是随机取 [lower, upper] 之间的一个数

lower：均匀分布下限

upper：均匀分布上限

## <a id='toc1_9_'></a>[权重初始化](#toc0_)

全0（不好）：所有权重都设置为0

随机初始化：权重服从均值为0，标准差为1的正态分布，如果初始值过大或过小，可能导致梯度爆炸或消失。

Xavier初始化：权重服从均值为0，标准差为2/n的正态分布，其中n是输入维度,通过修改标准差，让

He初始化：权重服从均值为0，标准差为2/sqrt(n)的正态分布，其中n是输入维度。针对ReLU激活函数效果更好。

He初始化和Xavier初始化都会向内挤压高斯分布，高斯分布有更加尖锐的峰值，其最高点要大得多。，这样的神经元更不容易饱和。

见神经网络与机器学习P86，里面写的：假设我们有⼀个有 nin 个输⼊权重的神经元。我们会使⽤均值为 0 标准差为 1/√nin 的高斯随机分布初始化这些权重。也就是说，我们会向下挤压⾼斯分布，让我们的神经元更不可能饱和

### <a id='toc1_9_1_'></a>[激活函数与推荐初始化方法](#toc0_)

激活函数	            推荐初始化方法

Sigmoid / Tanh	        Xavier/Glorot

ReLU / LeakyReLU	    He/Kaiming

默认用 Xavier（Sigmoid/Tanh）或 He（ReLU）初始化，避免全零初始化。

## <a id='toc1_10_'></a>[Hook 函数](#toc0_)

是在不改变主体的情况下，实现额外功能。由于 PyTorch 是基于动态图实现的，因此在一次迭代运算结束后，一些中间变量如非叶子节点的梯度和特征图，会被释放掉。在这种情况下想要提取和记录这些中间变量，就需要使用 Hook 函数。（比如之前在bcp中提取中间的特征图）



## <a id='toc1_11_'></a>[正则化](#toc0_)

L1，L2正则化，本质都是缩小权重，但是L1正则化对小权重的效果更明显，对大权重的效果弱于L2正则化，L2正则化对大权重的效果更明显。因此表现为L1让权重变得稀疏，L2让权重变得更加平滑。 也就是删小减大。具体可以见神经网络与机器学习P79。

L1正则化：给损失函数加上函数的绝对值之和，即：loss = loss + alpha * sum(abs(w))

$\mathcal{L} = \mathcal{L} + \alpha \cdot \sum |w|$

更规范的写法是

$\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{original}} + \alpha \cdot \|\mathbf{w}\|_1$

L2正则化：给损失函数加上函数的平方和，即：loss = loss + alpha * sum(w^2)

$\mathcal{L} = \mathcal{L} + \alpha \cdot \sum w^2$

更规范的写法是

$\mathcal{L}_{\text{total}} = \mathcal{L}_{\text{original}} + \alpha \cdot \|\mathbf{w}\|_2^2$

Dropout：随机让某些神经元不工作，让网络不要太过依赖于某个神经元，变相降低权重，防止过拟合，公式为：

$\mathcal{L} = \mathcal{L} + \text{dropout}(p) \cdot \sum_{i=1}^n \frac{1}{2}(1-p) \cdot \sigma(z_i)$




## <a id='toc1_12_'></a>[归一化](#toc0_)

常用且效果好的就是GN和BN，为组归一化和批归一化。归一化指的是将数据映射到[0,1]或[-1,1]的范围内，使得数据分布变得更加均匀，从而加速训练，也能使模型训练更加稳定。

GN：Group Normalization，组归一化，将输入分组，每组内的数据归一化，然后再将各组数据拼接起来，GN的思想是对输入进行分组，每组数据归一化，然后再拼接。其不依赖batch维度，可以适用于batchsize=1的任务，如检测和分割。公式为：

$$ \hat{x}_i =\gamma \frac{x_i - \mu_\mathcal{G}}{\sqrt{\sigma_\mathcal{G}^2 + \epsilon}} + \beta $$

其中$\hat{x}_i$是归一化后的输入，$\mu_\mathcal{G}$和$\sigma_\mathcal{G}^2$是组内均值和方差，$\gamma$和$\beta$是可学习的系数。

BN：Batch Normalization，批量归一化，批是指一批数据，通常为 mini-batch；标准化是处理后的数据服从$N(0,1)$的正态分布。BN的思想是对每一层的输入做归一化，使得数据分布的均值为0，方差为1。公式为：

$$y = \gamma \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} + \beta$$

其中$y$是归一化后的输出，$\gamma$和$\beta$是可学习的系数，$\mu_B$和$\sigma_B^2$是批内均值和方差。

GN和BN的区别：

- GN的分组操作使得每组数据归一化，而BN是对整个批数据归一化。
- GN的归一化公式中，$\gamma$和$\beta$是可学习的系数，可以适用于不同的输入。
- BN的归一化公式中，$\gamma$和$\beta$是固定不变的，只能适用于特定输入。

## <a id='toc1_13_'></a>[模型保存与加载](#toc0_)
### <a id='toc1_13_1_'></a>[基本方法](#toc0_)
#### <a id='toc1_13_1_1_'></a>[(1) 保存整个模型（结构+参数）](#toc0_)
```python
import torch
import torch.nn as nn

# 定义一个简单模型
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 2)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 保存模型
torch.save(model, 'model_entire.pth')  # 保存整个模型
```

#### <a id='toc1_13_1_2_'></a>[(2) 仅保存模型参数（推荐）](#toc0_)
```python
# 保存参数（体积更小）
torch.save(model.state_dict(), 'model_weights.pth')

# 加载时需先重建结构
new_model = SimpleModel()  # 必须与原模型结构相同
new_model.load_state_dict(torch.load('model_weights.pth'))
```

### <a id='toc1_13_2_'></a>[带优化器的保存（用于恢复训练）](#toc0_)
```python
# 保存
checkpoint = {
    'epoch': 10,
    'model_state': model.state_dict(),
    'optimizer_state': optimizer.state_dict(),
    'loss': loss_value
}
torch.save(checkpoint, 'checkpoint.pth')

# 加载
checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optimizer_state'])
epoch = checkpoint['epoch']
```

### <a id='toc1_13_3_'></a>[跨设备加载](#toc0_)
```python
# GPU保存 → CPU加载
model = torch.load('gpu_model.pth', map_location=torch.device('cpu'))

# 自动适配设备
model = SimpleModel()
model.load_state_dict(torch.load('weights.pth', map_location='cuda:0' if torch.cuda.is_available() else 'cpu'))
```

### <a id='toc1_13_4_'></a>[注意事项](#toc0_)
1. **文件扩展名**：建议使用 `.pth` 或 `.pt`  
2. **版本兼容性**：
   - 保存时指定PyTorch版本：`torch.save(..., _use_new_zipfile_serialization=True)`
   - 加载旧模型可能需兼容模式：`torch.load(..., pickle_module=pickle)`
3. **安全警告**：只加载可信来源的模型文件（可能包含恶意代码）

### <a id='toc1_13_5_'></a>[完整示例代码](#toc0_)
```python
# 保存示例
def save_model(model, path):
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': {'input_dim': 10, 'output_dim': 2}  # 保存结构配置
    }, path)

# 加载示例
def load_model(path):
    checkpoint = torch.load(path)
    model = SimpleModel(**checkpoint['config'])  # 根据配置重建模型
    model.load_state_dict(checkpoint['model_state_dict'])
    return model
```

### <a id='toc1_13_6_'></a>[常用文件结构](#toc0_)
```
project/
├── models/
│   ├── model_weights.pth    # 参数文件
│   └── checkpoint.pth      # 训练状态
└── train.py                # 训练/加载代码
```